In [1]:
import pandas as pd

In [2]:
wd='UMich_Sample_Data/'
file=(wd+'UMich-SampleData-PicksMadePNW-Jan3.csv')

wd1='data/'
file1=(wd1+'UMich-DataRequest3112022-LAX.csv')
file2=(wd1+'UMich-DataRequest3112022-PNW.csv')


In [11]:
b=pd.read_csv(file1)


In [53]:
b.columns

Index(['TIMESTAMP_UTC', 'PICK_TIMESTAMP_LOCAL', 'BOXBARCODE', 'BOXID',
       'ORDER_ID', 'BOXINTIME', 'BOXOUTTIME', 'PICKORDERSUBMITTIME',
       'PICKORDERSTARTTIME', 'PICKORDERENDTIME', 'DELIVERYDATE',
       'PICKORDERQTY', 'PICKEDQTY', 'PRODUCTBARCODE', 'AREANAME', 'ZONENAME',
       'SHELF', 'LEVEL', 'PICKS_DELETED', 'PRODUCTNAME'],
      dtype='object')

In [4]:
def combine(row):
    '''
    function that combines the product name, picktime and quantity into a list that is then put in a column
    '''
    pname=row['PRODUCTNAME']
    time=row['PICKCOMPLETETIME']
    qty=row['QTY']
    return [pname, time, qty]

In [5]:
def get_package_start_time(df):
    #trying to gather the box in time 
    
    '''
    takes in: a dataframe
    returns:  a dictionary of boxbarcodes and the start time
    
    '''
    
    start=df.groupby('BOXBARCODE')['PICKORDERSTARTTIME'].apply(list)
    start=start.reset_index()
    start['PICKORDERSTARTTIME']=start['PICKORDERSTARTTIME'].apply(lambda x: x[0])
    start['PICKORDERSTARTTIME']=pd.to_datetime(start['PICKORDERSTARTTIME'])

    start=start.set_index('BOXBARCODE')
    start=start.to_dict()
    #df=pd.from_dict(start)
        
    return start['PICKORDERSTARTTIME']
  

# returns dictionary with box ID's as keys and start times as values
#THIS IS NEEDED IN FINDING TIME DIFFERENCE BETWEEN TIME OF FIRST ITEM PACKED



In [79]:
def get_time_diff(boxname, lst,box_start_times):
    
    '''
    takes in: a list containing lists containing product name, time of stamp, and quantity
    
    returns: [Product name, the time difference, picktime]
    
    '''
    
    output_lst=[]  

    
    #sort product,picktime by picktime within list
    lst=sorted(lst, key=lambda x: pd.Timestamp(x[1]))
    
    #turning l=[(p1,t1),(p2,t2),(p3,t3)]
    #into new_l=[(p1,t0-t1),(p2,t1-t2),(p3,t2-t3)]
    
    
    for i in range(len(lst)):        
        # if its the first instance in the list, then we need to use the "pack box" or whatever it's called as our 
        #start time (our t0) in order to create a difference
        qty=lst[i][2]
        t1=pd.Timestamp(lst[i][1])
        if qty == 0:
            pass
        else:
            if i==0:
                #t0=box_start_times[boxname]
                #t0=pd.Timestamp(t0)
                #time_delta=t1-t0
                #time_delta=time_delta/qty

                time_delta=None  

            else:
                t0=lst[i-1][1]
                t0=pd.Timestamp(t0)
                time_delta=t1-t0
                time_delta=time_delta/qty
                

            try:
                output_lst.append([lst[i][0],time_delta.total_seconds(), t1, qty])
                if sum([t[1] for t in output_lst]) < 0:
                    print(boxname)
            except:
                output_lst.append([lst[i][0],None, t1, qty])
    

    return output_lst
    

In [48]:
def mean(lst):
    summ=0
    count=0
    for num in lst:
        if num==None:
            pass
        else:
            num=num.total_seconds()
            summ+=num
            count+=1
    return summ/count

Calculate distance travelled? 

In [77]:

def main3(file):
    import pandas as pd
    
    #~~~~~~~~~~~~~(MAY NEED TO ALTER HERE)~~~~~~~~~~~~~~~
    #loading file from csv into pandas 
    b=pd.read_csv(file)
    #~~~~~~~~~~~~~(MAY NEED TO ALTER HERE)~~~~~~~~~~~~~~~
    
    
    ###PREPROCESSING
    
    
    #Removing instances where item quantity is 0
    #b=b[b['QTY']!=0]
    
    #getting columns we need
    if 'QTY' in b.columns:
        if 'PICKCOMPLETETIME' in b.columns:      
            df=b[['PICKCOMPLETETIME','BOXBARCODE','PRODUCTNAME','QTY']]
        else:
            df=b[['PICK_TIMESTAMP_LOCAL','BOXBARCODE','PRODUCTNAME','QTY']]
            df=df.rename(columns={'PICK_TIMESTAMP_LOCAL':'PICKCOMPLETETIME'})
            
            #changing time zone to UTC 
            df['PICKCOMPLETETIME']=pd.to_datetime(df['PICKCOMPLETETIME'])  #convert to datetime object
            df['PICKCOMPLETETIME']= df['PICKCOMPLETETIME'].dt.tz_localize(tz='US/Pacific') #localize to pacific
            df['PICKCOMPLETETIME']= df['PICKCOMPLETETIME'].dt.tz_convert(tz='UTC') #change to utc time
            df['PICKCOMPLETETIME']= df['PICKCOMPLETETIME'].dt.tz_localize(None) #remove time zone

            
            

    else:
        if 'PICKCOMPLETETIME' in b.columns:
            df=b[['PICKCOMPLETETIME','BOXBARCODE','PRODUCTNAME','PICKEDQTY']]
            df=df.rename(columns={'PICKEDQTY':'QTY'})
        else:
            df=b[['PICK_TIMESTAMP_LOCAL','BOXBARCODE','PRODUCTNAME','PICKEDQTY']]
            df=df.rename(columns={'PICKEDQTY':'QTY', 'PICK_TIMESTAMP_LOCAL':'PICKCOMPLETETIME'})
                        #changing time zone to UTC 
            df['PICKCOMPLETETIME']=pd.to_datetime(df['PICKCOMPLETETIME'])
            df['PICKCOMPLETETIME']= df['PICKCOMPLETETIME'].dt.tz_localize(tz='US/Pacific')
            df['PICKCOMPLETETIME']= df['PICKCOMPLETETIME'].dt.tz_convert(tz='UTC')
            df['PICKCOMPLETETIME']= df['PICKCOMPLETETIME'].dt.tz_localize(None) #remove time zone

            

    
    #concatenating all relevant columns into a single list in a column
    df['concat']=df.apply(lambda x: combine(x), axis=1)
    
    #removing all other columns except barboxcode and concat
    df=df[['BOXBARCODE','concat']]

    #grouping by boxbarcode and cooncatenating all the items in 'concat' into a list so you have lsits of products
    df=df.groupby('BOXBARCODE')['concat'].apply(list)
    df=df.reset_index(name='[Product, PickTime, QTY]')
    
    
    
    #MAIN ANALYSIS
    #returns dictionary with box ID's as keys and start times as values
    box_start_times=get_package_start_time(b)

    y=df.apply(lambda x:get_time_diff(x['BOXBARCODE'], x['[Product, PickTime, QTY]'],box_start_times), axis=1)
    
    #turn into dataframe where each row is an instance
    reformatted=[]
    for biglst in y:
        for lst in biglst:
            reformatted.append(lst)
    output_df= pd.DataFrame(reformatted, columns=['ProductName', 'TimeDelta', 'PickTime', 'QTY'])

    #Creates a dicitonary where keys are product names and values are list of product times
    
    #formatting df for output
    output_df['PickTime']=output_df['PickTime'].apply(lambda x: x.date())
    results=output_df.groupby(['ProductName','PickTime']).sum()
    results=results.reset_index()
    results=results.rename(columns={'TimeDelta':'TotalPickTime', 'PickTime':'Date','QTY':'TotalQuantity'})
    
    return results
    

In [ ]:
results1=main3(file1)
#results1.to_csv('LA-Jan1-31-a.csv')
#results1.head()

In [149]:
results2=main3(file2)
results2.to_csv('PNW-Jan1-31.csv')
results2.head()

,ProductName,Date,TotalPickTime,TotalQuantity
0,PNW-15229,2022-01-16,13220.749999,368.0
1,PNW-15229,2022-01-17,16574.999998,519.0
2,PNW-15229,2022-01-18,14408.166665,388.0
3,PNW-15229,2022-01-19,12204.416666,418.0
4,PNW-15229,2022-01-23,2134.333333,136.0


In [199]:
s=b['PICK_TIMESTAMP_LOCAL'].head()
s=pd.to_datetime(s)
s.dt.tz_localize(tz='US/Pacific')


0   2022-01-02 12:27:08-08:00
1   2022-01-02 12:31:11-08:00
2   2022-01-02 12:23:02-08:00
3   2022-01-02 12:31:26-08:00
4   2022-01-02 12:22:05-08:00
Name: PICK_TIMESTAMP_LOCAL, dtype: datetime64[ns, US/Pacific]

In [190]:
b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1579589 entries, 0 to 1579588
Data columns (total 20 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   TIMESTAMP_UTC         1579589 non-null  object 
 1   PICK_TIMESTAMP_LOCAL  1579589 non-null  object 
 2   BOXBARCODE            1579216 non-null  object 
 3   BOXID                 1579589 non-null  int64  
 4   ORDER_ID              1579589 non-null  object 
 5   BOXINTIME             1579216 non-null  object 
 6   BOXOUTTIME            1577067 non-null  object 
 7   PICKORDERSUBMITTIME   1579589 non-null  object 
 8   PICKORDERSTARTTIME    1579571 non-null  object 
 9   PICKORDERENDTIME      1577440 non-null  object 
 10  DELIVERYDATE          1579589 non-null  object 
 11  PICKORDERQTY          1579589 non-null  float64
 12  PICKEDQTY             1579589 non-null  float64
 13  PRODUCTBARCODE        1579589 non-null  object 
 14  AREANAME              1579241 non-

## Results  Exploration

In [168]:
dict1=main3(file1)

In [25]:
results=main3(file)
results.to_csv('output.csv')

<ipython-input-24-d0a2a01a3af9>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['concat']=df.apply(lambda x: combine(x), axis=1)


In [26]:
results.head()


,ProductName,Date,TotalPickTime,TotalQuantity
0,PNW-15230,2022-01-02,92.000000,9.0
1,PNW-15230,2022-01-03,6281.666665,222.0
2,PNW-15230,2022-01-04,527.500000,12.0
3,PNW-20573,2022-01-02,76.000000,19.0
4,PNW-20573,2022-01-03,14366.000000,512.0


In [28]:
results.to_csv('output2.csv')

In [156]:
la=pd.read_csv(file1)
la.head()

,TIMESTAMP_UTC,PICK_TIMESTAMP_LOCAL,BOXBARCODE,BOXID,ORDER_ID,BOXINTIME,BOXOUTTIME,PICKORDERSUBMITTIME,PICKORDERSTARTTIME,PICKORDERENDTIME,DELIVERYDATE,PICKORDERQTY,PICKEDQTY,PRODUCTBARCODE,AREANAME,ZONENAME,SHELF,LEVEL,PICKS_DELETED,PRODUCTNAME
0,2022-01-03 15:00:29,2022-01-03 07:00:29,PRO_cdcd7e24-baa4-4555-9dd4-70b7ff39f090,2812779,cdcd7e24-baa4-4555-9dd4-70b7ff39f090,2021-12-28 01:43:30,2022-01-03 07:00:56,2021-12-28 01:43:30,2022-01-03 06:50:24,2022-01-03 07:00:56,2021-12-28 00:00:00,1.0,1.0,LA-20744,Line 02,Zone 13,B,5.0,True,LA-20744
1,2022-01-03 14:58:33,2022-01-03 06:58:33,PRO_cdcd7e24-baa4-4555-9dd4-70b7ff39f090,2812779,cdcd7e24-baa4-4555-9dd4-70b7ff39f090,2021-12-28 01:43:30,2022-01-03 07:00:56,2021-12-28 01:43:30,2022-01-03 06:50:24,2022-01-03 07:00:56,2021-12-28 00:00:00,1.0,1.0,LA-6240,Line 02,Zone 08,A,6.0,True,LA-6240
2,2022-01-03 14:59:00,2022-01-03 06:59:00,PRO_cdcd7e24-baa4-4555-9dd4-70b7ff39f090,2812779,cdcd7e24-baa4-4555-9dd4-70b7ff39f090,2021-12-28 01:43:30,2022-01-03 07:00:56,2021-12-28 01:43:30,2022-01-03 06:50:24,2022-01-03 07:00:56,2021-12-28 00:00:00,1.0,1.0,LA-6608,Line 02,Zone 09,A,7.0,True,LA-6608
3,2022-01-03 14:58:59,2022-01-03 06:58:59,PRO_81eb09c4-c1ea-4817-b3eb-14fe1da52183,2812643,81eb09c4-c1ea-4817-b3eb-14fe1da52183,2021-12-28 01:43:21,2022-01-03 07:00:11,2021-12-28 01:43:21,2022-01-03 06:49:47,2022-01-03 07:00:11,2021-12-28 00:00:00,1.0,1.0,LA-PMSHRMJERKYORIG,Line 02,Zone 11,C,8.0,True,LA-PMSHRMJERKYORIG
4,2022-01-03 14:57:46,2022-01-03 06:57:46,PRO_65aa3dcb-500b-4a4b-9033-9292f62d3d40,2811996,65aa3dcb-500b-4a4b-9033-9292f62d3d40,2021-12-28 01:42:33,2022-01-03 07:01:59,2021-12-28 01:42:33,2022-01-03 06:50:26,2022-01-03 07:01:59,2021-12-28 00:00:00,1.0,1.0,LA-5512,Line 02,Zone 07,B,1.0,True,LA-5512


In [165]:
b1=la[la['BOXBARCODE']=='PRO_a829db9c-51a8-49fc-86e7-04680468ac05'].copy()
b1.head()

,TIMESTAMP_UTC,PICK_TIMESTAMP_LOCAL,BOXBARCODE,BOXID,ORDER_ID,BOXINTIME,BOXOUTTIME,PICKORDERSUBMITTIME,PICKORDERSTARTTIME,PICKORDERENDTIME,DELIVERYDATE,PICKORDERQTY,PICKEDQTY,PRODUCTBARCODE,AREANAME,ZONENAME,SHELF,LEVEL,PICKS_DELETED,PRODUCTNAME
1431,2022-01-03 00:16:10,2022-01-02 16:16:10,PRO_a829db9c-51a8-49fc-86e7-04680468ac05,2822158,a829db9c-51a8-49fc-86e7-04680468ac05,2022-01-02 01:09:26,2022-01-02 16:21:08,2022-01-02 01:09:26,2022-01-02 16:09:12,2022-01-02 16:21:08,2022-01-02 00:00:00,1.0,1.0,LA-31609,Line 03,Zone 06,B,8.0,True,LA-31609
3841,2022-01-03 00:18:36,2022-01-02 16:18:36,PRO_a829db9c-51a8-49fc-86e7-04680468ac05,2822158,a829db9c-51a8-49fc-86e7-04680468ac05,2022-01-02 01:09:26,2022-01-02 16:21:08,2022-01-02 01:09:26,2022-01-02 16:09:12,2022-01-02 16:21:08,2022-01-02 00:00:00,1.0,1.0,LA-31659,Line 03,Zone 10,A,1.0,True,LA-31659
16328,2022-01-03 00:20:59,2022-01-02 16:20:59,PRO_a829db9c-51a8-49fc-86e7-04680468ac05,2822158,a829db9c-51a8-49fc-86e7-04680468ac05,2022-01-02 01:09:26,2022-01-02 16:21:08,2022-01-02 01:09:26,2022-01-02 16:09:12,2022-01-02 16:21:08,2022-01-02 00:00:00,1.0,1.0,LA-5264,Line 03,Zone 13,A,6.0,True,LA-5264
43864,2022-01-03 00:16:57,2022-01-02 16:16:57,PRO_a829db9c-51a8-49fc-86e7-04680468ac05,2822158,a829db9c-51a8-49fc-86e7-04680468ac05,2022-01-02 01:09:26,2022-01-02 16:21:08,2022-01-02 01:09:26,2022-01-02 16:09:12,2022-01-02 16:21:08,2022-01-02 00:00:00,1.0,1.0,LA-31597,Line 03,Zone 07,A,6.0,True,LA-31597
43867,2022-01-03 00:19:18,2022-01-02 16:19:18,PRO_a829db9c-51a8-49fc-86e7-04680468ac05,2822158,a829db9c-51a8-49fc-86e7-04680468ac05,2022-01-02 01:09:26,2022-01-02 16:21:08,2022-01-02 01:09:26,2022-01-02 16:09:12,2022-01-02 16:21:08,2022-01-02 00:00:00,1.0,1.0,LA-5163,Line 03,Zone 11,B,1.0,True,LA-5163
